In [1]:
from sentence_transformers import SentenceTransformer, models, InputExample
from torch.utils.data import DataLoader
from scipy.spatial.distance import cosine

In [2]:
model = SentenceTransformer("neuralmind/bert-base-portuguese-cased")

# 1. Fine-tuning

## 1.1 Preparação dos dados

In [7]:
import pandas as pd

arvore_proposicoes = pd.read_csv("data/arvore-proposicoes.csv")
df_proposicoes = pd.read_csv("data/proposicao-tema-completo-sem-duplicado.csv")

In [5]:
arvore_proposicoes

,numeroSequencia,nivel,codProposicao,codProposicaoReferenciada,codProposicaoRaiz,TipoReferencia
0,8,1,13033,13335,13335,Apensação
1,7,1,13227,24325,24325,Apensação
2,23,1,13240,24480,24480,Apensação
3,24,1,13270,24480,24480,Apensação
4,3,1,13322,164372,164372,Apensação
...,...,...,...,...,...,...
344601,4,1,2216834,1672171,1672171,Proposição Acessória
344602,5,1,2216872,796203,796203,Proposição Acessória
344603,6,1,2216873,796203,796203,Proposição Acessória
344604,8,1,2216874,1344458,1344458,Proposição Acessória


In [8]:
df_proposicoes

,codProposicao,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,imgArquivoTeorPDF,idTema
0,16357,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...,34
1,20464,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","_Alteração, Lei Federal, composição, (TRT), ex...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI...",34
2,20683,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,...,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\r\n Departament...,44
3,20857,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,...,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \r\nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA ...,41
4,26177,PL 4080/2001,Torna obrigatória a destinação de área para o ...,Obriga a destinação de área para estacionament...,"Obrigatoriedade, destinação, área, exclusivida...",\r\n \r\n807632679711411411145701111141161014...,57
...,...,...,...,...,...,...,...
51998,2265119,PL 5136/2020,Cria mecanismos para coibir a violência e a di...,...,"Criação, meios, repressão, violência política ...","PROJETO DE LEI Nº , DE 2020\r\n...",44
51999,2265120,PL 5137/2020,Revoga o artigo 246 da Lei 4737/65 de 15 de j...,...,"Alteração, Código Eleitoral (1965), revogação,...",CÂMARA DOS DEPUTADOS\r\nGabinete do Deputado A...,57
52000,2265123,PL 3877/2020,"Dispõe sobre o acolhimento, pelo Banco Central...",...,"Autorização, Banco Central do Brasil (BACEN), ...","Dispõe sobre o acolhimento, pelo Banco\r\...",40
52001,2265125,PL 5138/2020,Acrescenta parágrafo o art. 6º do Decreto-Lei ...,...,"Alteração, Código de Processo Penal, intimação...",C Â M A R A D O S D E P U T A D O S \r\n1 \r\n...,43
